# Agents and Tools

#### ChatGPT Functions

In [139]:
import sqlite3

conn = sqlite3.connect('db.sqlite')

In [140]:
def run_query(query:str):
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        conn.commit()
        return cursor.fetchall()
    except sqlite3.OperationalError as err:
        return f"The following error occurred: {str(err)}"
    
def describe_tables(tables_names):
    c = conn.cursor()
    tables = ', '.join("'" + table + "'" for table in tables_names)
    query = f"SELECT sql FROM sqlite_master WHERE type='table' and name in ({tables});"
    rows = c.execute(query) 
    formated = '\n'.join(row[0] for row in rows if row[0] is not None)

    print(
        "tables: \n",
        formated
    )

    return formated



In [141]:
from langchain.tools import Tool


run_query_tool = Tool.from_function(
    name="run_sqlite_query",
    description="""
    run a query on the sqlite database
    """,
    func=run_query,
)

describe_tables_tool = Tool.from_function(
    name="describe_tables",
    description="""
    Given a list of tables name, return the tables that are present in the database
    """,
    func=describe_tables,
)

In [142]:
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.agents import OpenAIFunctionsAgent, AgentExecutor
from dotenv import load_dotenv
from langchain.schema import SystemMessage
load_dotenv()


True

In [143]:
def list_list_tables():
    c = conn.cursor()
    c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    rows = c.fetchall() 
    return "\n".join(row[0] for row in rows if row[0] is not None)


In [144]:
chat = ChatOpenAI(
    model='gpt-3.5-turbo',
)

tables = list_list_tables()



chat_prompt = ChatPromptTemplate(
    messages =[
        SystemMessage(
            content=f"You are an AI that has access to SQLite database.\n The database has tables of:\n {tables}.\n Do not make any assumptions about the what tables exist or what colums exist. Instead use the 'describe_tables' function."
        ),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

In [145]:
tools = [run_query_tool, describe_tables_tool]

agent = OpenAIFunctionsAgent(
    llm=chat,
    prompt=chat_prompt,
    tools=tools,
)

agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    tools=tools,
)


In [146]:
agent_executor("How many users are in the database?")



> Entering new AgentExecutor chain...

Invoking: `describe_tables` with `users`


tables: 
 

Invoking: `run_sqlite_query` with `SELECT COUNT(*) FROM users`
responded: It seems that I do not have access to the table structure at the moment. Let me try again.

[(2000,)]There are 2000 users in the database.

> Finished chain.


{'input': 'How many users are in the database?',
 'output': 'There are 2000 users in the database.'}

In [147]:
agent_executor("How many users provided shipping address?")



> Entering new AgentExecutor chain...

Invoking: `describe_tables` with `users`


tables: 
 

Invoking: `describe_tables` with `addresses`
responded: It seems like I do not have access to the schema of the 'users' table. Let me try again.

tables: 
 

Invoking: `run_sqlite_query` with `SELECT COUNT(DISTINCT user_id) FROM addresses`
responded: The 'addresses' table contains the information about shipping addresses. Let me check how many users provided a shipping address by querying the 'addresses' table.

[(2000,)]There are 2000 users who provided a shipping address.

> Finished chain.


{'input': 'How many users provided shipping address?',
 'output': 'There are 2000 users who provided a shipping address.'}